In [1]:
%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3
from pathlib import Path
import pathlib
import numpy as np
import pandas as pd

# required to enable non-blocking interaction:
%gui qt5

## Pho's Custom Libraries:
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.function_helpers import function_attributes

# pyPhoPlaceCellAnalysis:
# NeuroPy (Diba Lab Python Repo) Loading
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.RachelDataSessionFormat import RachelDataSessionFormat
from neuropy.core.session.Formats.Specific.HiroDataSessionFormat import HiroDataSessionFormatRegisteredClass

## For computation parameters:
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths

# from PendingNotebookCode import _perform_batch_plot, _build_batch_plot_kwargs
from pyphoplacecellanalysis.General.Batch.NonInteractiveWrapper import batch_load_session, batch_extended_computations, SessionBatchProgress, batch_programmatic_figures, batch_extended_programmatic_figures
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme
from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import saveData, loadData
from pyphoplacecellanalysis.General.Batch.runBatch import BatchRun
from pyphoplacecellanalysis.General.Batch.runBatch import run_diba_batch

active_global_batch_result_filename='global_batch_result.pkl'
debug_print=True
""" 
from pyphoplacecellanalysis.General.Batch.runBatch import main, BatchRun, run_diba_batch, run_specific_batch

"""
global_data_root_parent_path = find_first_extant_path([Path(r'W:\Data'), Path(r'/media/MAX/Data'), Path(r'/Volumes/MoverNew/data'), Path(r'/home/halechr/turbo/Data')])
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"

## TODO: load the batch result initially:

## Build Pickle Path:
finalized_loaded_global_batch_result_pickle_path = Path(global_data_root_parent_path).joinpath(active_global_batch_result_filename).resolve() # Use Default

## Override:
# finalized_loaded_global_batch_result_pickle_path = Path('data/global_batch_run-2023-05-17.pkl').resolve()

if debug_print:
	print(f'finalized_loaded_global_batch_result_pickle_path: {finalized_loaded_global_batch_result_pickle_path}')
# try to load an existing batch result:
try:
	global_batch_run = loadData(finalized_loaded_global_batch_result_pickle_path, debug_print=debug_print)
	
except NotImplementedError:
	# Fixes issue with pickled POSIX_PATH on windows for path.
	posix_backup = pathlib.PosixPath # backup the PosixPath definition
	try:
		pathlib.PosixPath = pathlib.PurePosixPath
		global_batch_run = loadData(finalized_loaded_global_batch_result_pickle_path, debug_print=debug_print)
	finally:
		pathlib.PosixPath = posix_backup # restore the backup posix path definition
		
except (FileNotFoundError, TypeError):
	# loading failed
	print(f'Failure loading {finalized_loaded_global_batch_result_pickle_path}.')
	global_batch_run = None


Automatic pdb calling has been turned OFF
build_module_logger(module_name="Spike3D.pipeline"):
	 Module logger com.PhoHale.Spike3D.pipeline has file logging enabled and will log to EXTERNAL/TESTING/Logging/debug_com.PhoHale.Spike3D.pipeline.log
finalized_loaded_global_batch_result_pickle_path: /media/MAX/Data/global_batch_result.pkl
Loading loaded session pickle file results : /media/MAX/Data/global_batch_result.pkl... encountered exception 'BatchRun' object is not iterable while printing. Turning into a warning and continuing.
done.


In [5]:
global_batch_run = None

In [ ]:
# Convert the paths to work on the new system:
global_batch_run.change_global_root_path(global_data_root_parent_path)

batch_progress_df = global_batch_run.to_dataframe(expand_context=True) # all


good_only_batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=True)
good_only_batch_progress_df

# Get good sessions for use in the specific session processing notebook:

In [2]:
batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=False) # all
good_only_batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=True)
good_only_batch_progress_df

,format_name,animal,exper_name,session_name,context,basedirs,status,errors,n_long_laps,n_long_replays,n_short_laps,n_short_replays,is_ready
1,kdiba,gor01,one,2006-6-08_14-26-15,kdiba_gor01_one_2006-6-08_14-26-15,/media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-2...,SessionBatchProgress.COMPLETED,None,40,749,40,677,True
2,kdiba,gor01,one,2006-6-09_1-22-43,kdiba_gor01_one_2006-6-09_1-22-43,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22-43,SessionBatchProgress.COMPLETED,None,42,348,40,422,True
4,kdiba,gor01,one,2006-6-12_15-55-31,kdiba_gor01_one_2006-6-12_15-55-31,/media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-5...,SessionBatchProgress.COMPLETED,None,40,65,34,123,True
5,kdiba,gor01,one,2006-6-13_14-42-6,kdiba_gor01_one_2006-6-13_14-42-6,/media/MAX/Data/KDIBA/gor01/one/2006-6-13_14-42-6,SessionBatchProgress.COMPLETED,None,40,45,40,54,True
6,kdiba,gor01,two,2006-6-07_16-40-19,kdiba_gor01_two_2006-6-07_16-40-19,/media/MAX/Data/KDIBA/gor01/two/2006-6-07_16-4...,SessionBatchProgress.COMPLETED,None,40,661,40,1068,True
10,kdiba,gor01,two,2006-6-12_16-53-46,kdiba_gor01_two_2006-6-12_16-53-46,/media/MAX/Data/KDIBA/gor01/two/2006-6-12_16-5...,SessionBatchProgress.COMPLETED,None,40,118,40,77,True
12,kdiba,vvp01,one,2006-4-09_17-29-30,kdiba_vvp01_one_2006-4-09_17-29-30,/media/MAX/Data/KDIBA/vvp01/one/2006-4-09_17-2...,SessionBatchProgress.COMPLETED,None,63,57,44,38,True


In [17]:
## Get the list of sessions that are completely ready to process:
full_good_ready_to_process_sessions = list(good_only_batch_progress_df['context'].to_numpy())
full_good_ready_to_process_sessions
# Get good sessions for use in the specific session processing notebook:

[IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15')>,
 IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-09_1-22-43')>,
 IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-12_15-55-31')>,
 IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-13_14-42-6')>,
 IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6-07_16-40-19')>,
 IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6-12_16-53-46')>,
 IdentifyingContext<('kdiba', 'vvp01', 'one', '2006-4-09_17-29-30')>]

In [ ]:

print(",\n".join([ctx.get_initialization_code_string() for ctx in full_good_ready_to_process_sessions])) # List definitions

# [IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
# IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30')]

In [ ]:
print("\ncurr_context = ".join([ctx.get_initialization_code_string() for ctx in full_good_ready_to_process_sessions])) # Line definitions

# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30')

# Build `global_batch_run` pre-loading results (before execution)

In [ ]:

# global_batch_result = loadData('global_batch_result.pkl')
global_batch_run = run_diba_batch(global_data_root_parent_path, execute_all=False, extant_batch_run=global_batch_run, debug_print=False)
# print(f'global_batch_result: {global_batch_run}')
global_batch_run

# Run Batch Executions/Computations

In [8]:
import pandas as pd
from neuropy.core.epoch import Epoch

def _on_complete_success_execution_session(curr_session_context, curr_session_basedir, curr_active_pipeline):
	""" called when the execute_session completes like:
		`post_run_callback_fn_output = post_run_callback_fn(curr_session_context, curr_session_basedir, curr_active_pipeline)`
		
		Meant to be assigned like:
		, post_run_callback_fn=_on_complete_success_execution_session
	"""
	print(f'_on_complete_success_execution_session(curr_session_context: {curr_session_context}, curr_session_basedir: {str(curr_session_basedir)}, ...)')
	# print(f'curr_session_context: {curr_session_context}, curr_session_basedir: {str(curr_session_basedir)}')
	long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()
	long_session, short_session, global_session = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
	long_results, short_results, global_results = [curr_active_pipeline.computation_results[an_epoch_name]['computed_data'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]

	# Get existing laps from session:
	long_laps, short_laps, global_laps = [curr_active_pipeline.filtered_sessions[an_epoch_name].laps.as_epoch_obj() for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
	long_replays, short_replays, global_replays = [Epoch(curr_active_pipeline.filtered_sessions[an_epoch_name].replay.epochs.get_valid_df()) for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
	# short_laps.n_epochs: 40, long_laps.n_epochs: 40
	# short_replays.n_epochs: 6, long_replays.n_epochs: 8
	print(f'short_laps.n_epochs: {short_laps.n_epochs}, long_laps.n_epochs: {long_laps.n_epochs}')
	print(f'short_replays.n_epochs: {short_replays.n_epochs}, long_replays.n_epochs: {long_replays.n_epochs}')
	return {long_epoch_name:(long_laps, long_replays), short_epoch_name:(short_laps, short_replays)}


## I got it doing the bare-minimum loading and computations, so it should be ready to update the laps and constraint the placefields to those. Then we should be able to set up the replays at the same time.
# finally, we then finish by computing.

## Execute with the custom arguments.
active_computation_functions_name_whitelist=['_perform_baseline_placefield_computation',
                                        # '_perform_time_dependent_placefield_computation',
                                        '_perform_extended_statistics_computation',
                                        '_perform_position_decoding_computation', 
                                        # '_perform_firing_rate_trends_computation',
                                        '_perform_pf_find_ratemap_peaks_computation',
                                        # '_perform_time_dependent_pf_sequential_surprise_computation'
                                        # '_perform_two_step_position_decoding_computation',
                                        # '_perform_recursive_latent_placefield_decoding'
                                    ]
# active_computation_functions_name_whitelist=['_perform_baseline_placefield_computation']
global_batch_run.execute_all(force_reload=False, skip_extended_batch_computations=True, post_run_callback_fn=_on_complete_success_execution_session, **{'computation_functions_name_whitelist': active_computation_functions_name_whitelist, 'active_session_computation_configs': None}) # can override `active_session_computation_configs` if we want to set custom ones like only the laps.)

# 4m 39.8s

basedir: /media/MAX/Data/KDIBA/gor01/one/2006-6-07_11-26-53
active_data_mode_name: kdiba
Loading loaded session pickle file results : /media/MAX/Data/KDIBA/gor01/one/2006-6-07_11-26-53/loadedSessPickle.pkl... done.
Loading pickled pipeline success: /media/MAX/Data/KDIBA/gor01/one/2006-6-07_11-26-53/loadedSessPickle.pkl.
property already present in pickled version. No need to save.
using provided computation_functions_name_whitelist: ['_perform_baseline_placefield_computation', '_perform_extended_statistics_computation', '_perform_position_decoding_computation', '_perform_pf_find_ratemap_peaks_computation']
	 TODO: this will prevent recomputation even when the blacklist/whitelist or computation function definitions change. Rework so that this is smarter.
updating computation_results...
done.
	 TODO: this will prevent recomputation even when the blacklist/whitelist or computation function definitions change. Rework so that this is smarter.
updating computation_results...
done.
	 TODO: th

/home/halechr/repo/NeuroPy/neuropy/core/session/Formats/SessionSpecifications.py:140: UserWarning: WARNING: Optional File: /media/MAX/Data/KDIBA/gor01/one/2006-6-09_3-23-37/2006-6-09_3-23-37.dat does not exist. Continuing without it.
  warnings.warn(f'WARNING: Optional File: {an_optional_filepath} does not exist. Continuing without it.')


done.
Loading matlab import file results : /media/MAX/Data/KDIBA/gor01/one/2006-6-09_3-23-37/2006-6-09_3-23-37.position_info.mat... done.
Loading matlab import file results : /media/MAX/Data/KDIBA/gor01/one/2006-6-09_3-23-37/2006-6-09_3-23-37.spikes.mat... done.
Failure loading .position.npy. Must recompute.

Computing linear positions for all active epochs for session... Saving updated position results results : /media/MAX/Data/KDIBA/gor01/one/2006-6-09_3-23-37/2006-6-09_3-23-37.position.npy... 2006-6-09_3-23-37.position.npy saved
done.
	 force_recompute is True! Forcing recomputation of .interpolated_spike_positions.npy

Computing interpolate_spike_positions columns results : spikes_df... done.
	 Saving updated interpolated spike position results results : /media/MAX/Data/KDIBA/gor01/one/2006-6-09_3-23-37/2006-6-09_3-23-37.interpolated_spike_positions.npy... 2006-6-09_3-23-37.interpolated_spike_positions.npy saved
done.
Loading matlab import file results : /media/MAX/Data/KDIBA/gor01

/home/halechr/repo/NeuroPy/neuropy/core/epoch.py:358: UserWarning: curr_epochs already empty prior to any filtering
  warn(f'curr_epochs already empty prior to any filtering')
/home/halechr/repo/NeuroPy/neuropy/core/epoch.py:379: UserWarning: curr_epochs already empty prior to filtering by firing rate or minimum active units
  warn(f'curr_epochs already empty prior to filtering by firing rate or minimum active units')


	 curr_replays: 0
skip_save_on_initial_load is True so resultant pipeline will not be saved to the pickle file.
Applying session filter named "maze1"...
Constraining to epoch with times (start: 0.0, end: 528.0656055328436)
computing neurons mua for session...

Applying session filter named "maze2"...
Constraining to epoch with times (start: 528.0656055328436, end: 530.2439119999763)
computing neurons mua for session...

Applying session filter named "maze"...
Constraining to epoch with times (start: 0.0, end: 530.2439119999763)
computing neurons mua for session...

using provided computation_functions_name_whitelist: ['_perform_baseline_placefield_computation', '_perform_extended_statistics_computation', '_perform_position_decoding_computation', '_perform_pf_find_ratemap_peaks_computation']
due to whitelist, including only 4 out of 15 registered computation functions.
Recomputing active_epoch_placefields... using self.config.grid_bin_bounds_1D: (25.08263415708267, 259.85857952491784)
	

...

	 curr_replays: 70
skip_save_on_initial_load is True so resultant pipeline will not be saved to the pickle file.
Applying session filter named "maze1"...
Constraining to epoch with times (start: 0.0, end: 317.9822682701051)
computing neurons mua for session...

Applying session filter named "maze2"...
Constraining to epoch with times (start: 317.9822682701051, end: 510.9178815188352)
computing neurons mua for session...

Applying session filter named "maze"...
Constraining to epoch with times (start: 0.0, end: 510.9178815188352)
computing neurons mua for session...

using provided computation_functions_name_whitelist: ['_perform_baseline_placefield_computation', '_perform_extended_statistics_computation', '_perform_position_decoding_computation', '_perform_pf_find_ratemap_peaks_computation']
due to whitelist, including only 4 out of 15 registered computation functions.
Recomputing active_epoch_placefields... using self.config.grid_bin_bounds_1D: (9.691741837597057, 260.34381026785434)


/home/halechr/repo/NeuroPy/neuropy/core/session/Formats/SessionSpecifications.py:140: UserWarning: WARNING: Optional File: /media/MAX/Data/KDIBA/vvp01/two/2006-4-28_12-38-13/2006-4-26_13-51-50.eeg does not exist. Continuing without it.
  warnings.warn(f'WARNING: Optional File: {an_optional_filepath} does not exist. Continuing without it.')
/home/halechr/repo/NeuroPy/neuropy/core/session/Formats/SessionSpecifications.py:140: UserWarning: WARNING: Optional File: /media/MAX/Data/KDIBA/vvp01/two/2006-4-28_12-38-13/2006-4-26_13-51-50.dat does not exist. Continuing without it.
  warnings.warn(f'WARNING: Optional File: {an_optional_filepath} does not exist. Continuing without it.')
/home/halechr/repo/NeuroPy/neuropy/core/session/Formats/SessionSpecifications.py:140: UserWarning: WARNING: Optional File: /media/MAX/Data/KDIBA/vvp01/two/2006-4-28_12-38-13/2006-4-25_17-33-28.eeg does not exist. Continuing without it.
  warnings.warn(f'WARNING: Optional File: {an_optional_filepath} does not exist.

done.
Failure loading .position.npy. Must recompute.

Computing linear positions for all active epochs for session... Saving updated position results results : /media/MAX/Data/KDIBA/vvp01/two/2006-4-28_12-38-13/2006-4-28_12-38-13.position.npy... 2006-4-28_12-38-13.position.npy saved
done.
	 force_recompute is True! Forcing recomputation of .interpolated_spike_positions.npy

Computing interpolate_spike_positions columns results : spikes_df... done.
	 Saving updated interpolated spike position results results : /media/MAX/Data/KDIBA/vvp01/two/2006-4-28_12-38-13/2006-4-28_12-38-13.interpolated_spike_positions.npy... 2006-4-28_12-38-13.interpolated_spike_positions.npy saved
done.
Loading matlab import file results : /media/MAX/Data/KDIBA/vvp01/two/2006-4-28_12-38-13/2006-4-28_12-38-13.laps_info.mat... done.
setting laps object.
session.laps loaded successfully!
Loading matlab import file results : /media/MAX/Data/KDIBA/vvp01/two/2006-4-28_12-38-13/2006-4-28_12-38-13.replay_info.mat... done

/home/halechr/repo/NeuroPy/neuropy/core/session/Formats/SessionSpecifications.py:140: UserWarning: WARNING: Optional File: /media/MAX/Data/KDIBA/vvp01/two/2006-4-28_17-6-14/2006-4-28_12-38-13.eeg does not exist. Continuing without it.
  warnings.warn(f'WARNING: Optional File: {an_optional_filepath} does not exist. Continuing without it.')
/home/halechr/repo/NeuroPy/neuropy/core/session/Formats/SessionSpecifications.py:140: UserWarning: WARNING: Optional File: /media/MAX/Data/KDIBA/vvp01/two/2006-4-28_17-6-14/2006-4-28_12-38-13.dat does not exist. Continuing without it.
  warnings.warn(f'WARNING: Optional File: {an_optional_filepath} does not exist. Continuing without it.')
/home/halechr/repo/NeuroPy/neuropy/core/session/Formats/SessionSpecifications.py:140: UserWarning: WARNING: Optional File: /media/MAX/Data/KDIBA/vvp01/two/2006-4-28_17-6-14/2006-4-26_13-51-50.eeg does not exist. Continuing without it.
  warnings.warn(f'WARNING: Optional File: {an_optional_filepath} does not exist. Co

done.
Failure loading .position.npy. Must recompute.

Computing linear positions for all active epochs for session... Saving updated position results results : /media/MAX/Data/KDIBA/vvp01/two/2006-4-28_17-6-14/2006-4-28_17-6-14.position.npy... 2006-4-28_17-6-14.position.npy saved
done.
	 force_recompute is True! Forcing recomputation of .interpolated_spike_positions.npy

Computing interpolate_spike_positions columns results : spikes_df... done.
basedir: /media/MAX/Data/KDIBA/pin01/11-02_17-46-44
active_data_mode_name: kdiba
Loading loaded session pickle file results : /media/MAX/Data/KDIBA/pin01/11-02_17-46-44/loadedSessPickle.pkl... done.
Failure loading /media/MAX/Data/KDIBA/pin01/11-02_17-46-44/loadedSessPickle.pkl.
basedir: /media/MAX/Data/KDIBA/pin01/11-02_19-28-0
active_data_mode_name: kdiba
Loading loaded session pickle file results : /media/MAX/Data/KDIBA/pin01/11-02_19-28-0/loadedSessPickle.pkl... done.
Failure loading /media/MAX/Data/KDIBA/pin01/11-02_19-28-0/loadedSessPickle

In [9]:
# Save to file:
saveData(finalized_loaded_global_batch_result_pickle_path, global_batch_run) # Update the global batch run dictionary


Saving (file mode 'w+b') saved session pickle file results : /media/MAX/Data/global_batch_result.pkl... done.


In [ ]:
## Single Session:
_test_out = global_batch_run.execute_session(session_context=curr_sess_context, force_reload=True, skip_extended_batch_computations=True, computation_functions_name_whitelist =['_perform_baseline_placefield_computation'], active_session_computation_configs=None) # can override `active_session_computation_configs` if we want to set custom ones like only the laps.)
_test_out

# global_batch_run.execute_session(session_context=curr_sess_context, force_reload=True, skip_extended_batch_computations=True, **{'computation_functions_name_whitelist': ['_perform_baseline_placefield_computation'], 'active_session_computation_configs': None}) # can override `active_session_computation_configs` if we want to set custom ones like only the laps.)

# 23.5s

# 2023-05-15 - Resuming - Need to make sure laps and replays are calculated manually for all sessions. Ideally early on in the process so useless computations aren't peformed on them.


In [ ]:

full_good_dirs = [k for k, v in global_batch_run.session_batch_errors.items() if v is None]
bad_dirs = [k for k, v in global_batch_run.session_batch_errors.items() if v is not None]
full_good_dirs
bad_dirs

In [ ]:
global_batch_run.session_batch_status

In [ ]:
global_batch_run.session_batch_status
global_batch_run.session_batch_errors

In [ ]:
## Common function to be executed post successful load:


## Estimate the laps and the replays

## Constrain the computational periods to the computed laps

## Set up the short track with the same binning as the long one

## Perform the extended computations:
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import create_daily_programmatic_display_function_testing_folder_if_needed, session_context_to_relative_path
# 2023-01-* - Call extended computations to build `_display_short_long_firing_rate_index_comparison` figures:
extended_computations_include_whitelist=['long_short_fr_indicies_analyses', 'jonathan_firing_rate_analysis'] # do only specifiedl
newly_computed_values = batch_extended_computations(curr_active_pipeline, include_whitelist=extended_computations_include_whitelist, include_global_functions=True, fail_on_exception=True, progress_print=True, debug_print=False)
newly_computed_values



def _perform_post_compute_function(curr_active_pipeline):
	from neuropy.analyses.placefields import PfND
	from PendingNotebookCode import constrain_to_laps
	from PendingNotebookCode import compute_short_long_constrained_decoders

	curr_active_pipeline = constrain_to_laps(curr_active_pipeline)

	(long_one_step_decoder_1D, short_one_step_decoder_1D), (long_one_step_decoder_2D, short_one_step_decoder_2D) = compute_short_long_constrained_decoders(curr_active_pipeline, recalculate_anyway=True)
	long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()
	long_epoch_context, short_epoch_context, global_epoch_context = [curr_active_pipeline.filtered_contexts[a_name] for a_name in (long_epoch_name, short_epoch_name, global_epoch_name)]
	long_session, short_session, global_session = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
	long_results, short_results, global_results = [curr_active_pipeline.computation_results[an_epoch_name]['computed_data'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
	long_pf1D, short_pf1D, global_pf1D = long_results.pf1D, short_results.pf1D, global_results.pf1D
	long_pf2D, short_pf2D, global_pf2D = long_results.pf2D, short_results.pf2D, global_results.pf2D
	decoding_time_bin_size = long_one_step_decoder_1D.time_bin_size # 1.0/30.0 # 0.03333333333333333
	# Backup and replace loaded replays with computed ones:
	long_replays, short_replays, global_replays = [a_session.replace_session_replays_with_estimates(require_intersecting_epoch=None, debug_print=False) for a_session in [long_session, short_session, global_session]]
	# 3m 40.3s

	from pyphoplacecellanalysis.General.Mixins.ExportHelpers import create_daily_programmatic_display_function_testing_folder_if_needed, session_context_to_relative_path
	# 2023-01-* - Call extended computations to build `_display_short_long_firing_rate_index_comparison` figures:
	extended_computations_include_whitelist=['long_short_fr_indicies_analyses', 'jonathan_firing_rate_analysis'] # do only specifiedl
	newly_computed_values = batch_extended_computations(curr_active_pipeline, include_whitelist=extended_computations_include_whitelist, include_global_functions=True, fail_on_exception=True, progress_print=True, debug_print=False)
	newly_computed_values




# Load Pipeline

In [ ]:
# ==================================================================================================================== #
# Load Data                                                                                                            #
# ==================================================================================================================== #

import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QTableWidget, QTableWidgetItem
from pathlib import Path

class PathListWidget(QWidget):
    def __init__(self, paths):
        super().__init__()

        # Create a vertical layout for the widget
        layout = QVBoxLayout()
        self.setLayout(layout)

        # Create a table widget to display the paths and statuses
        table_widget = QTableWidget()
        table_widget.setColumnCount(2)
        table_widget.setHorizontalHeaderLabels(["Path", "Status"])
        table_widget.setRowCount(len(paths))

        # Add each path and status to the table
        for row, path in enumerate(paths):
            path_item = QTableWidgetItem(str(path))
            status_item = QTableWidgetItem("True" if path.exists() else "False")
            table_widget.setItem(row, 0, path_item)
            table_widget.setItem(row, 1, status_item)

        # Add the table widget to the layout
        layout.addWidget(table_widget)


# if __name__ == '__main__':
#     # Example usage
#     app = QApplication(sys.argv)
#     paths = [Path("/path/to/file1"), Path("/path/to/file2"), Path("/path/to/file3")]
#     widget = PathListWidget(paths)
#     widget.show()
#     sys.exit(app.exec_())
    


In [ ]:
paths = list(global_batch_run.session_batch_basedirs.values())
widget = PathListWidget(paths)
widget.show()

In [ ]:
%pdb off
# %%viztracer
basedir = local_session_paths_list[1] # NOT 3
print(f'basedir: {str(basedir)}')

# # Read if possible:
# saving_mode = PipelineSavingScheme.SKIP_SAVING
# force_reload = False

# Force write:
saving_mode = PipelineSavingScheme.OVERWRITE_IN_PLACE
force_reload = True

# ==================================================================================================================== #
# Load Pipeline                                                                                                        #
# ==================================================================================================================== #
# epoch_name_whitelist = ['maze']
epoch_name_whitelist = None
active_computation_functions_name_whitelist=['_perform_baseline_placefield_computation', '_perform_time_dependent_placefield_computation', '_perform_extended_statistics_computation',
                                        '_perform_position_decoding_computation', 
                                        '_perform_firing_rate_trends_computation',
                                        # '_perform_pf_find_ratemap_peaks_computation',
                                        '_perform_time_dependent_pf_sequential_surprise_computation'
                                        '_perform_two_step_position_decoding_computation',
                                        # '_perform_recursive_latent_placefield_decoding'
                                    ]
curr_active_pipeline = batch_load_session(global_data_root_parent_path, active_data_mode_name, basedir, epoch_name_whitelist=epoch_name_whitelist,
                                          computation_functions_name_whitelist=active_computation_functions_name_whitelist,
                                          saving_mode=saving_mode, force_reload=force_reload,
                                          skip_extended_batch_computations=True, debug_print=False, fail_on_exception=True)

In [ ]:
from neuropy.analyses.placefields import PfND
from PendingNotebookCode import constrain_to_laps
from PendingNotebookCode import compute_short_long_constrained_decoders

curr_active_pipeline = constrain_to_laps(curr_active_pipeline)

In [ ]:
(long_one_step_decoder_1D, short_one_step_decoder_1D), (long_one_step_decoder_2D, short_one_step_decoder_2D) = compute_short_long_constrained_decoders(curr_active_pipeline, recalculate_anyway=True)
long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()
long_epoch_context, short_epoch_context, global_epoch_context = [curr_active_pipeline.filtered_contexts[a_name] for a_name in (long_epoch_name, short_epoch_name, global_epoch_name)]
long_session, short_session, global_session = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_results, short_results, global_results = [curr_active_pipeline.computation_results[an_epoch_name]['computed_data'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_pf1D, short_pf1D, global_pf1D = long_results.pf1D, short_results.pf1D, global_results.pf1D
long_pf2D, short_pf2D, global_pf2D = long_results.pf2D, short_results.pf2D, global_results.pf2D
decoding_time_bin_size = long_one_step_decoder_1D.time_bin_size # 1.0/30.0 # 0.03333333333333333
# 3m 40.3s

In [ ]:
short_one_step_decoder_1D.pf.plot_ratemaps_1D()

In [ ]:
short_results.pf1D_Decoder.pf.plot_ratemaps_1D()

In [ ]:
short_results.pf2D_Decoder.pf.plot_ratemaps_2D()
short_one_step_decoder_2D.pf.plot_ratemaps_2D()

In [ ]:
curr_active_pipeline.save_pipeline(saving_mode=PipelineSavingScheme.TEMP_THEN_OVERWRITE) # AttributeError: 'PfND_TimeDependent' object has no attribute '_included_thresh_neurons_indx'
# TypeError: cannot pickle 'MplMultiTab' object

# 2023-03-16 - Explore passing in long/short decoders specifically to `perform_full_session_leave_one_out_decoding_analysis`

In [ ]:
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import BasePositionDecoder, BayesianPlacemapPositionDecoder
from pyphoplacecellanalysis.Analysis.Decoder.decoder_result import perform_full_session_leave_one_out_decoding_analysis
from pyphoplacecellanalysis.Analysis.Decoder.decoder_result import SurpriseAnalysisResult
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import build_neurons_color_map # for plot_short_v_long_pf1D_comparison

# Get existing long/short decoders from the cell under "# 2023-02-24 Decoders"
long_decoder, short_decoder = deepcopy(long_one_step_decoder_1D), deepcopy(short_one_step_decoder_1D)
assert np.all(long_decoder.xbin == short_decoder.xbin)

# Backup and replace loaded replays with computed ones:
long_replays, short_replays, global_replays = [a_session.replace_session_replays_with_estimates(require_intersecting_epoch=None, debug_print=False) for a_session in [long_session, short_session, global_session]]

### Need to prune to only the cells active in both epochs ahead of time:
# Prune to the shared aclus in both epochs (short/long):
long_shared_aclus_only_decoder, short_shared_aclus_only_decoder = [BasePositionDecoder.init_from_stateful_decoder(a_decoder) for a_decoder in (long_decoder, short_decoder)]
shared_aclus, (long_shared_aclus_only_decoder, short_shared_aclus_only_decoder), long_short_pf_neurons_diff = BasePositionDecoder.prune_to_shared_aclus_only(long_shared_aclus_only_decoder, short_shared_aclus_only_decoder)
n_neurons = len(shared_aclus)
# for plotting purposes, build colors only for the common (present in both, the intersection) neurons:
neurons_colors_array = build_neurons_color_map(n_neurons, sortby=None, cmap=None)
print(f'{n_neurons = }, {neurons_colors_array.shape =}')
# from viztracer import VizTracer

# with VizTracer(output_file=f"viztracer_{get_now_time_str()}-full_session_LOO_decoding_analysis.json", min_duration=200, tracer_entries=3000000, ignore_frozen=True) as tracer:
long_results_obj = perform_full_session_leave_one_out_decoding_analysis(global_session, original_1D_decoder=long_shared_aclus_only_decoder, decoding_time_bin_size = 0.025, cache_suffix = '_long', perform_cache_load=False) # , perform_cache_load=False
short_results_obj = perform_full_session_leave_one_out_decoding_analysis(global_session, original_1D_decoder=short_shared_aclus_only_decoder, decoding_time_bin_size = 0.025, cache_suffix = '_short', perform_cache_load=False) # , perform_cache_load=False

# 4m 22.4s

In [ ]:
## 2023-05-01 - Test my interneuron hypothesis by looking at interneurons across the long/short divide


# 1. Interneuron identity should be best determinant of active environment.

# 2. Do people know anything about interneurons that activate in a given environment? For example, to interneurons activate alongside pyramidal cells when replaying a given environment?

# 3. 



## 2023-04-13 - Shuffled Surprise
""" 
Relevant Functions:
`perform_full_session_leave_one_out_decoding_analysis`:
	`perform_leave_one_aclu_out_decoding_analysis`:	from pyphoplacecellanalysis.Analysis.Decoder.decoder_result import perform_leave_one_aclu_out_decoding_analysis
	`_analyze_leave_one_out_decoding_results`: from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.DefaultComputationFunctions import _analyze_leave_one_out_decoding_results
"""


In [ ]:
# 2023-04-18 - Refactored into decoder_result
result, result_df, result_df_grouped = long_results_obj.result, long_results_obj.result_df, long_results_obj.result_df_grouped

In [ ]:
curr_active_pipeline

In [ ]:
curr_active_pipeline.global_computation_results

In [ ]:
from pyphoplacecellanalysis.Analysis.Decoder.decoder_result import DiagnosticDistanceMetricFigure

## Render the internactive slider that allows selecting the timebin index to debug
n_timebins = np.sum(long_results_obj.all_epochs_num_epoch_time_bins)
timebinned_neuron_info = long_results_obj.timebinned_neuron_info
result = long_results_obj.new_result
active_fig_obj, update_function = DiagnosticDistanceMetricFigure.build_interactive_diagnostic_distance_metric_figure(long_results_obj, timebinned_neuron_info, result)
active_fig_obj.integer_slider(n_timebins=n_timebins, update_func=update_function)

In [ ]:

# 1. Get set of cells active in a given time bin, for each compute the surprise of its placefield with the leave-one-out decoded posterior.

# 2. From the remainder of cells (those not active), randomly choose one to grab the placefield of and compute the surprise with that and the same posterior.

# Expectation: The cells that are included in the time bin are expected to have a lower surprise (be less correlated with) the posterior.

In [ ]:
from pyphoplacecellanalysis.Pho2D.PyQtPlots.Extensions.pyqtgraph_helpers import build_pyqtgraph_epoch_indicator_regions # Add session indicators to pyqtgraph plot
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import _helper_add_long_short_session_indicator_regions, plot_long_short_expected_vs_observed_firing_rates

from pyphocorehelpers.print_helpers import generate_html_string
# Long Short
# TODO 2023-04-18 - Can Refactor in terms of `plot_long_short_any_values`?

import pyphoplacecellanalysis.External.pyqtgraph as pg
# 'time_bin_indices': valid_time_bin_indicies, 'posterior_to_pf_mean_surprise': one_left_out_posterior_to_pf_surprises_mean, 'posterior_to_scrambled_pf_mean_surprise': one_left_out_posterior_to_scrambled_pf_surprises_mean}


In [ ]:


win, plots_tuple, legend = plot_long_short_expected_vs_observed_firing_rates(long_results_obj, short_results_obj)
long_epoch_indicator_region_items, short_epoch_indicator_region_items = _helper_add_long_short_session_indicator_regions(win, long_epoch=curr_active_pipeline.filtered_epochs[long_epoch_name], short_epoch=curr_active_pipeline.filtered_epochs[short_epoch_name])

In [ ]:


def plot_long_short_surprise_difference_plot(long_results_obj, short_results_obj):
	""" captures `curr_active_pipeline` """
	# Private Subfunctions _______________________________________________________________________________________________ #
	def _subfn_add_difference_plot_series(win, plots, result_df_grouped, series_suffix, **kwargs):
		""" captures nothing
		modifies `plots` """
		x=result_df_grouped.time_bin_centers.to_numpy()
		y=result_df_grouped['surprise_diff'].to_numpy()
		series_id_str = f'difference_{series_suffix}'
		plots[series_id_str] = win.plot(x=x, y=y, name=series_id_str, alpha=0.5, **kwargs) #  symbolBrush=pg.intColor(i,6,maxValue=128) , symbol=curr_symbol, symbolBrush=cell_color_symbol_brush[unit_IDX]

	# BEGIN FUNCTION BODY ________________________________________________________________________________________________ #

	# make a separate symbol_brush color for each cell:
	# cell_color_symbol_brush = [pg.intColor(i,hues=9, values=3, alpha=180) for i, aclu in enumerate(long_results_obj.original_1D_decoder.neuron_IDs)] # maxValue=128
	# All properties in common:
	win = pg.plot() # PlotWidget
	win.setWindowTitle('Long Sanity Check - Leave-one-out Custom Surprise Plot')
	# legend_size = (80,60) # fixed size legend
	legend_size = None # auto-sizing legend to contents
	legend = pg.LegendItem(legend_size, offset=(-1,0)) # do this instead of # .addLegend
	legend.setParentItem(win.graphicsItem())

	plots = {}
	label_prefix_list = ['normal', 'scrambled']
	long_short_symbol_list = ['t', 'o'] # note: 's' is a square. 'o', 't1': triangle pointing upwards0

	# Use mean time_bin and surprise for each epoch
	# plots['normal'] = win.plot(x=valid_time_bin_indicies, y=one_left_out_posterior_to_pf_surprises_mean, pen=None, symbol='t', symbolBrush=pg.intColor(1,6,maxValue=128), name=f'normal', alpha=0.5) #  symbolBrush=pg.intColor(i,6,maxValue=128) , symbol=curr_symbol, symbolBrush=cell_color_symbol_brush[unit_IDX]
	# plots['scrambled'] = win.plot(x=valid_time_bin_indicies, y=one_left_out_posterior_to_scrambled_pf_surprises_mean, pen=None, symbol='t', symbolBrush=pg.intColor(2,6,maxValue=128), name=f'scrambled', alpha=0.5) #  symbolBrush=pg.intColor(i,6,maxValue=128) , symbol=curr_symbol, symbolBrush=cell_color_symbol_brush[unit_IDX]

	# curr_surprise_difference = one_left_out_posterior_to_scrambled_pf_surprises_mean - one_left_out_posterior_to_pf_surprises_mean

	# x=valid_time_bin_indicies
	# y=curr_surprise_difference
	# x=result_df_grouped.time_bin_indices.to_numpy()

	
	_subfn_add_difference_plot_series(win, plots, long_results_obj.result_df_grouped, series_suffix='_long', **dict(pen=None, symbol='t', symbolBrush=pg.intColor(2,6,maxValue=128), clickable=True, hoverable=True, hoverSize=7))

	_subfn_add_difference_plot_series(win, plots, short_results_obj.result_df_grouped, series_suffix='_short', **dict(pen=None, symbol='o', symbolBrush=pg.intColor(3,6,maxValue=128), clickable=True))

	# dict(pen=None, symbol='t', symbolBrush=pg.intColor(2,6,maxValue=128))


	# x=result_df_grouped.time_bin_centers.to_numpy()
	# y=result_df_grouped['surprise_diff'].to_numpy()
	# plots['difference'] = win.plot(x=x, y=y, pen=None, symbol='t', symbolBrush=pg.intColor(2,6,maxValue=128), name=f'difference', alpha=0.5) #  symbolBrush=pg.intColor(i,6,maxValue=128) , symbol=curr_symbol, symbolBrush=cell_color_symbol_brush[unit_IDX]

	# long_results_obj.result, long_results_obj.result_df, long_results_obj.result_df_grouped

	# short_results_obj.result, short_results_obj.result_df, short_results_obj.result_df_grouped


	for k, v in plots.items():
		legend.addItem(v, f'{k}')

	win.graphicsItem().setLabel(axis='left', text='Normal v. Random - Surprise (Custom)')
	win.graphicsItem().setLabel(axis='bottom', text='time')

	win.showGrid(True, True)  # Show grid for reference

	# Emphasize the y=0 crossing by drawing a horizontal line at y=0
	vline = pg.InfiniteLine(pos=0, angle=0, movable=False, pen=pg.mkPen(color='w', width=2, style=pg.QtCore.Qt.DashLine))
	win.addItem(vline)

	# Add session indicators to pyqtgraph plot
	long_epoch = curr_active_pipeline.filtered_epochs[long_epoch_name]
	short_epoch = curr_active_pipeline.filtered_epochs[short_epoch_name]
	long_epoch_indicator_region_items, short_epoch_indicator_region_items = _helper_add_long_short_session_indicator_regions(win, long_epoch, short_epoch)

	# epoch_linear_region, epoch_region_label = build_pyqtgraph_epoch_indicator_regions(win, t_start=curr_active_pipeline.filtered_epochs[long_epoch_name].t_start, t_stop=curr_active_pipeline.filtered_epochs[long_epoch_name].t_stop, epoch_label='long', **dict(pen=pg.mkPen('#0b0049'), brush=pg.mkBrush('#0099ff42'), hoverBrush=pg.mkBrush('#fff400'), hoverPen=pg.mkPen('#00ff00')))
	# epoch_linear_region, epoch_region_label = build_pyqtgraph_epoch_indicator_regions(win, t_start=curr_active_pipeline.filtered_epochs[short_epoch_name].t_start, t_stop=curr_active_pipeline.filtered_epochs[short_epoch_name].t_stop, epoch_label='short', **dict(pen=pg.mkPen('#490000'), brush=pg.mkBrush('#f5161659'), hoverBrush=pg.mkBrush('#fff400'), hoverPen=pg.mkPen('#00ff00')))

	i_str = generate_html_string('i', color='white', bold=True)
	j_str = generate_html_string('j', color='red', bold=True)
	title_str = generate_html_string(f'JSD(p_x_given_n, pf[{i_str}]) - JSD(p_x_given_n, pf[{j_str}]) where {j_str} non-firing')
	win.setTitle(title_str)

	win.setWindowTitle('Long Sanity Check - Leave-one-out Custom Surprise Plot - JSD')

	return win, plots

win, plots = plot_long_short_surprise_difference_plot(long_results_obj, short_results_obj)

In [ ]:
def _helper_make_scatterplot_clickable(main_scatter_plot, enable_hover:bool=False):
	""" pyqtgraph """
	# Highlights the hovered spikes white:
	# main_scatter_plot.addPoints(hoverable=True,
	# 	# hoverSymbol=vtick, # hoverSymbol='s',
	# 	hoverSize=7, # default is 5
	# 	)


	## Clickable/Selectable Spikes:
	# Will make all plots clickable
	clickedPen = pg.mkPen('#DDD', width=2)
	lastClicked = []
	def _test_scatter_plot_clicked(plot, points):
		global lastClicked
		for p in lastClicked:
			p.resetPen()
		print("clicked points", points)
		for p in points:
			p.setPen(clickedPen)
		lastClicked = points

	main_scatter_clicked_connection = main_scatter_plot.sigClicked.connect(_test_scatter_plot_clicked)

	## Hoverable Spikes:
	if enable_hover:
		def _test_scatter_plot_hovered(plt, points, ev):
			# sigHovered(self, points, ev)
			print(f'_test_scatter_plot_hovered(plt: {plt}, points: {points}, ev: {ev})')
			if (len(points) > 0):
				curr_point = points[0]
				# self.
				# curr_point.index
		main_scatter_hovered_connection = main_scatter_plot.sigHovered.connect(_test_scatter_plot_hovered)
	else:
		main_scatter_hovered_connection = None

	return lastClicked, clickedPen, (main_scatter_hovered_connection, main_scatter_clicked_connection)


In [ ]:
a_plot = plots['difference__long']  # PlotDataItem 
a_curve = a_plot.curve # PlotCurveItem 

In [ ]:
a_plot.sigPointsClicked
# a_plot.sigPointsHovered

In [ ]:
curve.curve.setClickable(True)

In [ ]:
# Will make all plots clickable
clickedPen = pg.mkPen('#DDD', width=2)
lastClicked = []
def _test_scatter_plot_clicked(plot, points):
	global lastClicked
	for p in lastClicked:
		p.resetPen()
	print("clicked points", points)
	for p in points:
		p.setPen(clickedPen)
	lastClicked = points

main_scatter_clicked_connection = a_plot.sigClicked.connect(_test_scatter_plot_clicked)

In [ ]:
lastClicked, clickedPen, (main_scatter_hovered_connection, main_scatter_clicked_connection) = _helper_make_scatterplot_clickable(a_plot)

In [ ]:
# Need to map from time_bin_indicies to times
a_plot = plots['difference'] # PlotDataItem 
# a_plot.setLa
# win.graphicsItem().setLabel(axis='left', text='Normal v. Random - Surprise (Custom)')

# # Set the plot title with a LaTeX formula
# title = pg.LabelItem(justify='center')
# title.setText(r'<font size="4">JSD Surprise Diff: $\fn{JSD}{\matr{pf}_{i}, \Pr{\vec{x}_{t}|\vec{n}_{t}}} - \fn{JSD}{\matr{pf}_{j}, \Pr{\vec{x}_{t}|\vec{n}_{t}}}$</font>')
# win.addItem(title)

# win.graphicsItem().setLabel(axis='top', text=r'$\fn{JSD}{\matr{pf}_{i}, \Pr{\vec{x}_{t}|\vec{n}_{t}}} - \fn{JSD}{\matr{pf}_{j}, \Pr{\vec{x}_{t}|\vec{n}_{t}}}$')

In [ ]:
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import export_pyqtgraph_plot # works pretty well seemingly

export_pyqtgraph_plot(win)
# pg.setConfigOption('leftTitle', 'MathText')
# win.setTitle(r'JSD(p_x_given_n, pf[<font size="4"><b><span style="color:red;">i</span></b></font>]) - JSD(p_x_given_n, pf[<font size="4"><b>j</b></font>]) where <font size="4"><b>j</b></font> non-firing')

# win.setTitle(generate_html_string(f'JSD(p_x_given_n, pf[{i_str}]) - JSD(p_x_given_n, pf[{j_str}]) where {j_str} non-firing', font_size=8))

# r'$\fn{JSD}{\matr{pf}_{i}, \Pr{\vec{x}_{t}|\vec{n}_{t}}} - \fn{JSD}{\matr{pf}_{j}, \Pr{\vec{x}_{t}|\vec{n}_{t}}}$'
# title_str

In [ ]:
from pyqtgraph.GraphicsScene import exportDialog

# active_item = active_fig_obj.win
active_item = active_fig_obj.plot_dict['curr_cell_pf_curve']['plot_item']
exportDialog = exportDialog.ExportDialog(active_item.scene())
exportDialog.show(active_item)

In [ ]:
active_fig_obj.plot_dict['curr_cell_pf_curve']['plot_item']

$\fn{JSD}{\matr{pf}_{i}, \Pr{\vec{x}_{t}|\vec{n}_{t}}} - \fn{JSD}{\matr{pf}_{j}, \Pr{\vec{x}_{t}|\vec{n}_{t}}}$

In [ ]:
win.setBackground
win.setWindowTitle
win.setBackgroundBrush
win.setXRange
win.setYRange

"Show X Grid"
"Show Y Grid"


In [ ]:
visual_config = dict(pen=pg.mkPen('#fff'), brush=pg.mkBrush('#f004'), hoverBrush=pg.mkBrush('#fff4'), hoverPen=pg.mkPen('#f00'))

In [ ]:
curr_active_pipeline.filtered_epochs[short_epoch_name].t_stop

In [ ]:
curr_active_pipeline.filtered_epochs[long_epoch_name].start_end_times

## Pre 2023-04-13

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib qt

from pyphoplacecellanalysis.Analysis.Decoder.decoder_result import plot_kourosh_activity_style_figure

from neuropy.core.neurons import NeuronType
# # Include only pyramidal aclus:
# print(f'all shared_aclus: {len(shared_aclus)}\nshared_aclus: {shared_aclus}')
# shared_aclu_neuron_type = long_session.neurons.neuron_type[np.isin(long_session.neurons.neuron_ids, shared_aclus)]
# assert len(shared_aclu_neuron_type) == len(shared_aclus)
# # Find only the aclus that are pyramidal:
# is_shared_aclu_pyramidal = (shared_aclu_neuron_type == NeuronType.PYRAMIDAL)
# pyramidal_only_shared_aclus = shared_aclus[is_shared_aclu_pyramidal]
# print(f'num pyramidal_only_shared_aclus: {len(pyramidal_only_shared_aclus)}\npyramidal_only_shared_aclus: {pyramidal_only_shared_aclus}')


## Drop Pyramidal but don't use only shared aclus:
all_aclus = deepcopy(long_session.neurons.neuron_ids)
neuron_type = long_session.neurons.neuron_type
assert len(neuron_type) == len(all_aclus)
# Find only the aclus that are pyramidal:
is_aclu_pyramidal = (neuron_type == NeuronType.PYRAMIDAL)
pyramidal_only_all_aclus = all_aclus[is_aclu_pyramidal]
print(f'num pyramidal_only_all_aclus: {len(pyramidal_only_all_aclus)}\npyramidal_only_all_aclus: {pyramidal_only_all_aclus}')

is_aclu_interneuron = (neuron_type == NeuronType.INTERNEURONS)
interneuron_only_all_aclus = all_aclus[is_aclu_interneuron]
print(f'num interneuron_only_all_aclus: {len(interneuron_only_all_aclus)}\ninterneurons: {interneuron_only_all_aclus}')

# app, win, plots, plots_data = plot_kourosh_activity_style_figure(long_results_obj, long_session, shared_aclus, epoch_idx=5, callout_epoch_IDXs=[0,1,2,3], skip_rendering_callouts=True)
# app, win, plots, plots_data = plot_kourosh_activity_style_figure(long_results_obj, long_session, pyramidal_only_shared_aclus, epoch_idx=2, callout_epoch_IDXs=[0,4], skip_rendering_callouts=False)
app, win, plots, plots_data = plot_kourosh_activity_style_figure(long_results_obj, long_session, pyramidal_only_all_aclus, epoch_idx=6, callout_epoch_IDXs=[2,4,6], skip_rendering_callouts=False)

In [ ]:

# Use the Jupyter Index Thing
from pyphoplacecellanalysis.Analysis.Decoder.decoder_result import DiagnosticDistanceMetricFigure

## Render the internactive slider that allows selecting the timebin index to debug
n_timebins = np.sum(long_results_obj.all_epochs_num_epoch_time_bins)
timebinned_neuron_info = long_results_obj.timebinned_neuron_info
result = long_results_obj.new_result
active_fig_obj, update_function = DiagnosticDistanceMetricFigure.build_interactive_diagnostic_distance_metric_figure(long_results_obj, timebinned_neuron_info, result)
active_fig_obj.integer_slider(n_timebins=n_timebins, update_func=update_function)

In [ ]:
active_fig_obj.export()

In [ ]:
app, win, plots, plots_data = plot_kourosh_activity_style_figure(long_results_obj, long_session, pyramidal_only_all_aclus, epoch_idx=11, callout_epoch_IDXs=[0,1,2, 3, 4, 5], skip_rendering_callouts=False)

# 2023-04-13 Show Surprise 

In [ ]:
import pyphoplacecellanalysis.External.pyqtgraph as pg
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import plot_long_short, plot_long_short_any_values, plot_long_short_expected_vs_observed_firing_rates, _helper_add_long_short_session_indicator_regions
# plot_long_short(long_results_obj, short_results_obj)

In [ ]:
plot_long_short_any_values(long_results_obj=long_results_obj, short_results_obj=short_results_obj)

In [ ]:
plot_long_short_expected_vs_observed_firing_rates(long_results_obj=long_results_obj, short_results_obj=short_results_obj, limit_aclus=[89]) # 4, 89, 28, 97

In [ ]:
x_fn = lambda a_results_obj: a_results_obj.all_epochs_decoded_epoch_time_bins_mean[:,0]
# y_fn = lambda a_results_obj: a_results_obj.all_epochs_all_cells_one_left_out_posterior_to_scrambled_pf_surprises_mean
y_fn = lambda a_results_obj: a_results_obj.all_epochs_all_cells_one_left_out_posterior_to_pf_surprises_mean
# y_fn = lambda a_results_obj: a_results_obj.all_epochs_computed_one_left_out_posterior_to_pf_surprises

# (time_bins, neurons), (epochs, neurons), (epochs)
# all_epochs_computed_one_left_out_posterior_to_pf_surprises, all_epochs_computed_cell_one_left_out_posterior_to_pf_surprises_mean, all_epochs_all_cells_one_left_out_posterior_to_pf_surprises_mean
win, plots_tuple, legend = plot_long_short_any_values(long_results_obj, short_results_obj, x=x_fn, y=y_fn, limit_aclus=[20])

# 2023-04-13 - Find Good looking epochs:

In [ ]:
%matplotlib qt
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.DecoderPredictionError import plot_decoded_epoch_slices

laps_plot_tuple = plot_decoded_epoch_slices(long_results_obj.active_filter_epochs, long_results_obj.all_included_filter_epochs_decoder_result, global_pos_df=global_session.position.df, variable_name='lin_pos', xbin=long_results_obj.original_1D_decoder.xbin,
                                                        name='stacked_epoch_slices_long_results_obj', debug_print=False, debug_test_max_num_slices=32)
curr_viz_params, _curr_plot_data, _curr_plots, _curr_ui_container = laps_plot_tuple
_curr_plots.axs

In [ ]:
# [23, 27, 29, ]
[16, 17, 18, 20, 21, 22, 23, 25, 26, 29]


# 2023-03-28 - Playing around with older computations/visualizations from the `_display_short_long_firing_rate_index_comparison` era:


2023-04-20 - Encountered issue with the replays in session '2006-6-08_14-26-15' where they are duplicated exactly twice, like the first half of the rows are legitimate entries and the second half are directly repeated versions of the first with the only difference appearing to be the 'epoch_id' column changes from 1 to 2. 'rel_id' column seems incorrect but different for some reason. It must be how the MATLAB script exports the values.

Also when I'm looking at only the `short_session.replay` there are many non-2 'epoch_id' values, which is strange. 

TODO: It could have something to do with Jonathan's code maybe? Because the 'replay_r' and 'replay_p' columns he added are different. SEEMS FALSE. It's this way even without running Jonathan's code, although the values might have been saved later?
	Also flat_replay_idx jumps from 689 to 1087 at the transition from epoch_id 1 to 2

UPDATE: the 'replay_r' and 'replay_p' columns aren't from Joanthan, they're in the original .replay_info.mat that's imported!


# Plot long|short firing rate index using 'long_short_fr_indicies_analyses':

In [ ]:
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import create_daily_programmatic_display_function_testing_folder_if_needed, session_context_to_relative_path
# 2023-01-* - Call extended computations to build `_display_short_long_firing_rate_index_comparison` figures:
extended_computations_include_whitelist=['long_short_fr_indicies_analyses', 'jonathan_firing_rate_analysis'] # do only specifiedl
newly_computed_values = batch_extended_computations(curr_active_pipeline, include_whitelist=extended_computations_include_whitelist, include_global_functions=True, fail_on_exception=True, progress_print=True, debug_print=False)
newly_computed_values

In [ ]:
# fig_save_parent_path = Path(r'E:\Dropbox (Personal)\Active\Kamran Diba Lab\Results from 2023-01-20 - LongShort Firing Rate Indicies')
## Get the output path (active_session_figures_out_path) for this session (and all of its filtered_contexts as well):
active_identifying_session_ctx = curr_active_pipeline.sess.get_context() # 'bapun_RatN_Day4_2019-10-15_11-30-06'
figures_parent_out_path = create_daily_programmatic_display_function_testing_folder_if_needed()
active_session_figures_out_path = session_context_to_relative_path(figures_parent_out_path, active_identifying_session_ctx)
print(f'curr_session_parent_out_path: {active_session_figures_out_path}')
active_session_figures_out_path.mkdir(parents=True, exist_ok=True) # make folder if needed
curr_active_pipeline.display('_display_short_long_firing_rate_index_comparison', curr_active_pipeline.sess.get_context(), fig_save_parent_path=active_session_figures_out_path)
# plt.close() # closes the current figure

In [ ]:
# curr_active_pipeline.perform_specific_computation(computation_functions_name_whitelist=['_perform_short_long_firing_rate_analyses'], fail_on_exception=True, debug_print=False) # fail_on_exception MUST be True or error handling is all messed up 
long_short_fr_indicies_analysis_results = curr_active_pipeline.global_computation_results.computed_data['long_short_fr_indicies_analysis']
x_frs_index, y_frs_index = long_short_fr_indicies_analysis_results['x_frs_index'], long_short_fr_indicies_analysis_results['y_frs_index'] # use the all_results_dict as the computed data value
active_context = long_short_fr_indicies_analysis_results['active_context']
long_short_fr_indicies_analysis_results

# Other Programmatic Figures

In [ ]:
batch_extended_programmatic_figures(curr_active_pipeline=curr_active_pipeline)

In [ ]:
batch_programmatic_figures(curr_active_pipeline=curr_active_pipeline)

In [ ]:
# Output the 2D placefields for my presentation
from neuropy.core.neuron_identities import PlotStringBrevityModeEnum
from neuropy.plotting.ratemaps import BackgroundRenderingOptions

In [ ]:

# '_display_2d_placefield_result_plot_ratemaps_2D'
# %matplotlib inline
%matplotlib qt
# curr_active_pipeline.display('_display_2d_placefield_result_plot_ratemaps_2D', long_epoch_context) # MatplotlibRenderPlots


In [ ]:

# long_pf2D.plot_occupancy()
_out = short_one_step_decoder_2D.pf.plot_ratemaps_2D(included_unit_neuron_IDs=[2,4,5], bg_rendering_mode=BackgroundRenderingOptions.EMPTY, use_special_overlayed_title=False, missing_aclu_string_formatter=None, debug_print=False, brev_mode=PlotStringBrevityModeEnum.NONE)

## Single column output: subplots=(None, 9)


In [ ]:
# %%capture
active_display_to_pdf_fn(curr_active_pipeline, curr_display_function_name='_display_1d_placefields', debug_print=False) # 🟢✅ Now seems to be working and saving to PDF!! Still using matplotlib.use('Qt5Agg') mode and plots still appear.

In [ ]:
long_shared_aclus_only_decoder.pf.plot_occupancy()

In [ ]:
%matplotlib qt
long_one_step_decoder_2D.pf.plot_occupancy()

In [ ]:
long_one_step_decoder_2D.pf.plot_ratemaps_2D()

In [ ]:
# curr_active_pipeline.display('_display_short_long_pf1D_comparison', active_session_configuration_context=global_epoch_context, single_figure=False)

short_one_step_decoder_1D

In [ ]:
%matplotlib inline
active_identifying_session_ctx = curr_active_pipeline.sess.get_context() # 'bapun_RatN_Day4_2019-10-15_11-30-06'

long_single_cell_pfmap_processing_fn = None
short_single_cell_pfmap_processing_fn = None

# long_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: 0.5 * pfmap # flip over the y-axis
# short_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: -0.5 * pfmap # flip over the y-axis

# pad = 1
# long_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: (0.5 * pfmap) + (0.5*pad) # shift the baseline up by half
# short_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: (-0.5 * pfmap * pad) + (0.5*pad) # flip over the y-axis, shift the baseline down by half

# pad = 1
# long_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: (0.5 * pfmap * pad) + (0.5*pad) # shift the baseline up by half
# short_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: (0.5 * pfmap * pad) + (0.5*pad) # flip over the y-axis, shift the baseline down by half
# long_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: (0.5 * pfmap * pad) # shift the baseline up by half
# short_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: (0.5 * pfmap * pad) # flip over the y-axis, shift the baseline down by half


# long_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: (1.0 * pfmap * pad) # shift the baseline up by half
# short_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: (-1.0 * pfmap * pad) + (1.0*pad) # this does not work and results in short being fully filled. I think this is because the fill_between gets reversed since everything is below baseline

sort_idx = None

out = curr_active_pipeline.display('_display_short_long_pf1D_comparison', active_identifying_session_ctx, single_figure=False, debug_print=False, fignum='Short v Long pf1D Comparison',
                                   long_kwargs={'sortby': sort_idx, 'single_cell_pfmap_processing_fn': long_single_cell_pfmap_processing_fn},
                                   short_kwargs={'sortby': sort_idx, 'single_cell_pfmap_processing_fn': short_single_cell_pfmap_processing_fn, 'curve_hatch_style': {'hatch':'///', 'edgecolor':'k'}},
                                  )

# ax = out.axes[0]

In [ ]:
sort_idx = np.argsort(product_overlap_scalars_df.prod_overlap.to_numpy())[::-1] # the `[::-1]` term reverses the array, which by defaul is returned in ascending order and we want descending
sort_idx

In [ ]:


curr_ratemap = long_one_step_decoder_1D.pf.ratemap
curr_ratemap.get_sort_indicies()
# .pf1D.ratemap

# 2023-04-27 - Idea: Candidate Replay "Quality" Metric
Observed that I visually distinguish "good" replays from bad ones based on their mostly-monotonically increasing nature.

#### They don't have to:
	Span the whole track
	Start or end at an end-cap
	Increase linearly

#### The algorithm must:
	tolerate ocasional jumps in an otherwise linear sequence
	allow sequences to "start" only halfway through. They should extract out the coherent sequence

#### Ideas:
	- just a linear fit
	- a monotonicity check using some sort of cumulative sum over the differences in position.
	- "radon transform"?